In [1]:
search_for = 136
start_from = 0
threads = 10
thread = 2
log_freq = 10000



#in this cell, for a given lattice, we compute all forbidden nodes, 
#i.e. those nodes whose Voronoi cell is at a distance less than doubled covering radius

#An* represented in m=n+1 as sum x_i=0 with x_i\equiv to each other mod m. 
#Forbidden nodes will be recorded w.r.t the basis (-n,1,1,..) and permutations

n=5 #lattice dim
m=n+1 #space dim




R = sqrt(n*(n+1)*(n+2)/12) #covering radius

#first we generate all nodes with norm <=4R, and with norm<=2R
t=1 #.3 #can increase a bit for random from non-forbidden
short = []
vshort = []
K = ceil(t*4*R)
K2 = (t*4*R)^2
Kv = (2*R)^2

indexes = [] #arrays of integers between -K and K having the same residue modulo m 
for rem in range(m):
    res = []
    for i in range(-K,K+1):
        if i%m==rem:
            res.append(i)
    indexes.append(res)

for c in range(0,K+1):
    for coefs in mrange_iter([indexes[c%m] for i in range(m-2)]):
        p = [c]+list(coefs)+[-c-sum(coefs)]
        is_sorted = True
        for i in range(m-1):
            if p[i]<p[i+1]:
                is_sorted = False
                break
        if is_sorted:
            norm2 = sum((p[i])^2 for i in range(m))
            if norm2>0:
                if norm2 <= Kv:
                    vshort.append(p)
                if norm2 <= K2:
                    short.append(p)           

print("very short:",len(vshort))
print("short:",len(short))


from time import time
tm = time()


#defining the cone arising due to symmetry - it suffices to construct Voronoi cell in it
ineqs = [] 
for i in range(m-1):
    toadd = [0]*(m+1)
    toadd[i+1]=1
    toadd[i+2]=-1
    ineqs.append(toadd) 
    
ineqs += [[0]+[1]*m,[0]+[-1]*m] #we are in the hyperplane where all coords sum to zero

#now Voronoi cell planes
for p in vshort: #suffices to look at nodes at most 2R
    ineqs.append([sum((p[i])^2 for i in range(m))]+[-2*p[i] for i in range(m)])

V = Polyhedron(ieqs=ineqs)
    
print(time()-tm)    

### orthogonal projection affine operator onto (improper) non-empty polytope 
zerom = matrix(m)
def projp(P):
    nv = P.n_vertices()
    z = vector(P.vertices()[0])
    if nv==1:
        return zerom,z
    A = matrix([vector(P.vertices()[i])-z for i in range(1,nv)]).transpose()
    return A*A.pseudoinverse(),z


forb = []

for d in range(0,n): #dim of face
    for ff in V.faces(d):
        proj = projp(ff)
        for p in short:
            if p not in forb:
                x = vector([t/2 for t in p]) #dist from 1/2 of a node to the Voronoi cell equals 1/2 of the dist between cells at zero and at the node
                proj_pnt = proj[0]*(x-proj[1])+proj[1]
                if proj_pnt in ff.as_polyhedron():
                    dist = (x-proj_pnt).norm()
                    if dist<R:
                        forb.append(p)

print("forbidden:",len(forb))

#expand by the basis (-n,1,1,..) and permutations
def basis_coefs(x):
    y = x[0:(m-1)]
    sy = sum(y)
    return [(-sy-y[i])/m for i in range(m-1)]

### forbd = forb with possible permutations
forbd = {}
forbdv = []
for x in forb:
    for t in Permutations(list(x)):
        c = vector(basis_coefs(t))
        if str(c) not in forbd:
            forbd[str(c)]=0
            if sum(c)>=0:
                forbdv.append(c)
print("forbidden without symmetries:",len(forbd))
###





fn = 0
for x in forb:
    norm2 = sum(t^2 for t in x)
    if norm2>fn:
        fn = norm2
print("square L2 norm of forbidden:",fn)


small = 2 #max abs value of a small coefficient
sc = []
for c in mrange_iter([range(-small,small+1) for i in range(n)]):
    if sum(c)>=0:
        sc.append(vector(c))
sc.sort(key=lambda i: sum(x*x for x in i)) #abs(x) in place of x*x seems a bit slower
sc = sc[1:] #removing zero

print("small:",len(sc))

order = [(-1)^i*(1/4+i/2)-1/4 for i in range(1000)] #[0,-1,1,-2,2,...]

def check_sublattice(basis,target=Infinity): 
    A = matrix(basis) #rows
    #A = A.LLL() #benchmark to see if helps
    A = A.transpose() #columns

#     Ad = abs(A.det())
#     if Ad==0 or Ad>=target:
#         return Infinity
        
    for coefs in sc:
        if str(A*coefs) in forbd:
            return Infinity

    Ai = A.inverse()
    Ai_norms = [sum(t^2 for t in Ai[i]) for i in range(n)]
    M = []
    Mp = 1
    Mps = 1
    for i in range(n):
        mi = floor(sqrt(fn*Ai_norms[i]))
        M.append(mi)
        Mp *= (2*mi+1)
        Mps *= (2*min(small,mi)+1)
        
    if max(M)>small: #otherwise already considered
        if Mp-Mps<len(forbd): #faster using A, otherwise A_inv
            for coefs in mrange_iter([order[0:(2*M[i]+1)] for i in range(n)]):
                if sum(coefs)>=0 and max(abs(x) for x in coefs)>small:
                    if str(A*vector(coefs)) in forbd:
                        return Infinity
        else: #check if A_inv times forbidden is not in Z^n
            for x in forbdv:
                y = Ai*x
                allint = True
                for t in y:
                    if t!=floor(t):
                        allint = False
                        break
                if allint:
                    return Infinity   
    
    return 0#Ad



#random/exhaustive sublattice search, skip to the cell starting with good for search using non-forbidden generating vectors of small norm
#each type of a matrix is the numbers on the diagonal
maxd = search_for #bound for diagonal entries
mindet = search_for #minimum determinant
maxdet = search_for #maximum determinant

def all_types(m,d1,d2,n):
    res = []
    if n==1:
        for k in range(d1,min(d2,m)+1):
            res.append([k])
        return res
    for k in range(1,min(d2,m)+1):
        prev = all_types(m,ceil(d1/k),floor(d2/k),n-1)
        for x in prev:
            res.append([k]+x)
    return res

types = all_types(maxd,mindet,maxdet,n)

print("types:",len(types))

def ind_max(t): #given type, lists maxima of indexes to define entries above the diagonal
    return [t[i]^i for i in range(1,n)]


allc = 0
for t in types:
    allc += prod(ind_max(t))
print("all cases:",allc)

from random import randint
def rand_ind(t): #given type, return random indexes
    maxs = ind_max(t)
    return [randint(0,maxs[i]-1) for i in range(n-1)]

def sublat_mat(t,ind): #given type and indexes, return sublattice basis
    res = matrix(n)
    for i in range(n):
        res[i,i] = t[i]
    for i in range(1,n):
        tt = -floor(t[i]/2)
        indc = ind[i-1]
        for j in range(i):
            res[j,i] = tt + (indc % t[i])
            indc = floor(indc / t[i])
    return res

def exh_search():
    tm = time()
    bestv = Infinity
    best = []
    allcnt = 0
    for tt in types:
        print("type:",tt,prod(tt))
        cnt = 0
        maxinds = ind_max(tt)
        cM = prod(maxinds)
        print("cases of this type:",cM)
        for i in range(cM):
            allcnt += 1
            if allcnt%(log_freq*threads)==0:
                print(allcnt,(time()-tm)/3600*(allc-allcnt)/(log_freq*threads))
                tm=time()
            if allcnt>=start_from and allcnt%threads==thread:
                inds = []
                l = i
                for j in range(n-1):
                    inds.append(l%maxinds[j])
                    l=floor(l/maxinds[j])
                mat = sublat_mat(tt,inds)
                cs = check_sublattice(mat)
                if cs==0:
                    cs = prod(tt)
                    if cs<bestv:
                        print("sublattice basis:")
                        print(mat)
                        bestv = cs
                        best = [mat]
                    elif cs==bestv:
                        best.append(mat)
        print("best so far:",len(best))
    print("all cases:",allcnt)
    print("best:",len(best))
    return best





best = exh_search()




very short: 5
short: 39
0.08878850936889648
forbidden: 31
forbidden without symmetries: 1984
square L2 norm of forbidden: 264
small: 1752
types: 175
all cases: 1048119951
type: [1, 1, 1, 1, 136] 136
cases of this type: 342102016
100000 36.35396858275134
200000 61.94696540051413
300000 55.11978555174779
400000 53.616459551077114
500000 55.38454542680589
600000 57.10866870590709
700000 73.50937522179136
800000 78.11857156045106
900000 77.27716060135289
1000000 80.44390235385991
1100000 93.60160188681252
1200000 113.88970422450092
1300000 38.69584296359028
1400000 101.55796691020541
1500000 115.33828848911698
1600000 114.3080922712319
1700000 105.89318935520222
1800000 113.21588326206648
1900000 110.58901176374944
2000000 107.45892782352801
2100000 105.85803695366441
2200000 106.094077875953
2300000 115.39441709980218
2400000 109.85706825489102
2500000 93.11803140956604
2600000 34.31254074541084
2700000 104.66200779016422
2800000 100.31294666308237
2900000 100.08502351385069
3000000 100.6

29400000 105.4021634608765
29500000 104.84605244346496
29600000 102.97842414828239
29700000 104.94624440050531
29800000 106.9424787808644
29900000 104.7960427338168
30000000 65.61426905052066
30100000 81.58025557947205
30200000 112.11087381343043
30300000 117.15813760853004
30400000 93.00352987185383
30500000 58.90665525856739
30600000 112.60876819757146
30700000 116.4250660822551
30800000 113.57998003819846
30900000 107.10668639363982
31000000 117.61454260348067
31100000 111.64688724235924
31200000 105.99478001437733
31300000 107.15051671126037
31400000 102.91950175136041
31500000 20.853418166856528
31600000 108.19101162000806
31700000 103.76199029026327
31800000 107.69246475985906
31900000 111.67068280947004
32000000 106.83562545840975
32100000 106.7074193610125
32200000 111.19786073080088
32300000 115.2628233710136
32400000 109.07101600119853
32500000 69.48814015274294
32600000 88.68368226532915
32700000 121.11887064235674
32800000 110.02648388905304
32900000 111.578659617169
330000

59200000 42.9687099523755
59300000 91.74110535606947
59400000 95.20944882024794
59500000 94.2710982160878
59600000 100.86781254700178
59700000 96.99609503646782
59800000 100.45348775928531
59900000 103.10406345801319
60000000 34.22477600632058
60100000 112.23867619583784
60200000 102.47896384186377
60300000 105.61256635566507
60400000 104.28190453205823
60500000 110.73833580782711
60600000 107.32236082040866
60700000 106.98413200634268
60800000 93.64240283815745
60900000 50.229619359469886
61000000 106.97250778342499
61100000 105.92651905618357
61200000 101.6207840028352
61300000 102.36757602476987
61400000 95.44926309915306
61500000 100.85042801746737
61600000 89.36110819756348
61700000 29.063676029898552
61800000 98.25683607161695
61900000 100.71941357977963
62000000 100.36843934018322
62100000 101.20243292319768
62200000 102.49496630342313
62300000 101.02938225677744
62400000 106.9820946454412
62500000 33.67813739731815
62600000 111.66426409947164
62700000 107.58042530462629
6280000

89300000 61.45808231317697
89400000 51.54358977623403
89500000 86.6938786049252
89600000 92.10434860693488
89700000 86.95378372932441
89800000 86.42963560446178
89900000 75.13054771872822
90000000 57.027117581338814
90100000 90.06005491841876
90200000 92.31173381782577
90300000 96.94650401435585
90400000 96.62938866215998
90500000 100.8470302739331
90600000 97.32594680719149
90700000 102.34911778989259
90800000 103.73436104009953
90900000 104.93934420858514
91000000 99.71257795831475
91100000 104.2244328450993
91200000 95.73745475587108
91300000 42.359434565553315
91400000 96.4865382589866
91500000 99.8764899326348
91600000 96.46880497107698
91700000 91.7035507227985
91800000 80.34187839044257
91900000 42.74444087674135
92000000 92.54665421388766
92100000 94.19819234550641
92200000 93.04993524609763
92300000 89.61879556346165
92400000 75.71782621933873
92500000 61.26327069983082
92600000 104.4107836687941
92700000 95.47225786491778
92800000 102.08228215878655
92900000 108.7289699186670

118900000 96.31364887132459
119000000 91.54645501183683
119100000 63.450213317486636
119200000 53.02614545157341
119300000 87.88133169390649
119400000 82.70080197685779
119500000 49.82420113992119
119600000 65.92232012010469
119700000 84.45842900588103
119800000 95.58505329475652
119900000 49.51204204215115
120000000 73.28371896973955
120100000 91.464664353429
120200000 91.2315987516944
120300000 90.85831414999473
120400000 96.16913177665029
120500000 95.61531875867641
120600000 91.426756903596
120700000 97.94530971292572
120800000 100.66008279059605
120900000 104.35225813141851
121000000 106.1121821146173
121100000 100.42495915177189
121200000 99.74569985655623
121300000 97.83255758447196
121400000 95.96918728812953
121500000 92.08857795144002
121600000 92.1618183400571
121700000 31.878396897452724
121800000 93.97939650918866
121900000 95.33460570196395
122000000 64.69612638836466
122100000 52.57014543751142
122200000 87.27761733043455
122300000 100.41142362869695
122400000 48.0780925

...

952800000 0.6108004604548554
952900000 1.0827659220645494
953000000 1.7629094579678504
953100000 1.4769784075068555
953200000 1.355108387609213
953300000 1.7228598386322684
953400000 0.9949709903462589
953500000 0.8489836294930306
953600000 2.782320218880353
953700000 1.9311238651344103
953800000 2.233128033654196
953900000 2.397481346226096
954000000 1.1087570582549373
954100000 0.5920235426534408
954200000 1.6867061612207663
954300000 1.3929087536306441
954400000 1.4264636172146408
954500000 1.538860712954545
954600000 1.3566170342258912
954700000 0.5522010527075277
954800000 1.8019608082099463
954900000 1.3747924437418482
best so far: 0
type: [1, 2, 1, 17, 4] 136
cases of this type: 2515456
955000000 1.5787766910121335
955100000 1.2742287223113644
955200000 1.1916238750943482
955300000 1.068276673746209
955400000 1.0817786442277324
955500000 1.3306895689078828
955600000 1.068669279120404
955700000 1.401622846828399
955800000 1.313727238407443
955900000 1.3078103346635799
956000000 1

979100000 0.47121457198640443
best so far: 0
type: [1, 2, 17, 1, 4] 136
cases of this type: 147968
979200000 0.31825437221586905
979300000 0.35116792824763526
best so far: 0
type: [1, 2, 17, 2, 2] 136
cases of this type: 73984
979400000 0.3209639559012532
best so far: 0
type: [1, 2, 17, 4, 1] 136
cases of this type: 36992
best so far: 0
type: [1, 2, 34, 1, 2] 136
cases of this type: 36992
best so far: 0
type: [1, 2, 34, 2, 1] 136
cases of this type: 18496
best so far: 0
type: [1, 2, 68, 1, 1] 136
cases of this type: 9248
979500000 0.3088888736627519
best so far: 0
type: [1, 4, 1, 1, 34] 136
cases of this type: 5345344
979600000 3.151380311962155
979700000 2.4876245981379728
979800000 1.9363446823645256
979900000 1.2340411564829328
980000000 1.2929100665807682
980100000 2.278915885533724
980200000 1.8979995007808435
980300000 1.3189290030711909
980400000 1.8547595280938745
980500000 1.361492515153187
980600000 1.109581149720335
980700000 1.191801349773572
980800000 1.4771917620385342
98

998400000 0.8368814744214713
998500000 1.305347683003715
998600000 1.3372300442397396
998700000 0.8929458425271948
998800000 1.350448666227533
998900000 1.3733834379165986
999000000 0.9080509539161961
999100000 1.283127163060108
999200000 1.3497613511308393
999300000 0.9973276439718664
999400000 0.9738233696644493
999500000 1.3068053348368858
999600000 1.1228364008264522
999700000 0.8482079936835487
999800000 1.2812684180216245
999900000 1.1934404136564951
1000000000 0.9248304908365548
1000100000 1.3570116893331945
1000200000 1.3507547950078724
1000300000 0.9004545875836432
1000400000 1.3626497977813516
1000500000 0.24251946964226762
1000600000 0.24284713404200675
1000700000 0.24213180208939844
1000800000 0.24069901591734888
1000900000 0.2395796316662583
1001000000 0.23887797167231442
1001100000 0.24473545581783054
1001200000 0.23929746321481202
1001300000 0.2388830785642134
1001400000 0.2373711961705069
1001500000 0.24193017195873853
1001600000 0.23774207004163464
1001700000 0.2369337

1025300000 0.39976121207232673
1025400000 0.39390374361966846
1025500000 0.46239439497769547
1025600000 0.589442318435351
1025700000 0.2862533544582963
1025800000 0.28366544819449585
1025900000 0.3283144626388622
1026000000 0.3370220101131042
1026100000 0.1909502842709886
1026200000 0.45430522357128267
1026300000 0.2914462868227899
1026400000 0.22231731320177783
1026500000 0.552414730782842
1026600000 0.2795322312667012
1026700000 0.16535725985154445
1026800000 0.29890650161915155
1026900000 0.28311573701529696
1027000000 0.18945643020685846
1027100000 0.3117080378584193
1027200000 0.28018946804922223
1027300000 0.20072252017632589
1027400000 0.2511533300962336
1027500000 0.3104001942189932
1027600000 0.2365442084915949
1027700000 0.23529024556864828
best so far: 0
type: [2, 1, 1, 17, 4] 136
cases of this type: 1257728
1027800000 0.3121700344014525
1027900000 0.35746082520665196
1028000000 0.464986291180749
1028100000 0.2767036554212094
1028200000 0.25447534111933573
1028300000 0.22299

best so far: 0
type: [2, 34, 1, 2, 1] 136
cases of this type: 272
best so far: 0
type: [2, 34, 2, 1, 1] 136
cases of this type: 136
best so far: 0
type: [2, 68, 1, 1, 1] 136
cases of this type: 68
best so far: 0
type: [4, 1, 1, 1, 34] 136
cases of this type: 1336336
1045400000 0.044346584276189406
1045500000 0.05200186932424704
1045600000 0.06164044422141512
1045700000 0.047479420970113415
1045800000 0.04549401706573963
1045900000 0.03674300564171804
1046000000 0.029854523062564267
1046100000 0.0103581686265707
1046200000 0.029227859967788723
1046300000 0.02964488240706788
1046400000 0.03547729930941462
1046500000 0.028948033548321327
1046600000 0.028082937949761418
best so far: 0
type: [4, 1, 1, 2, 17] 136
cases of this type: 668168
1046700000 0.027029588636233406
1046800000 0.01563179224563175
1046900000 0.013688268154627746
1047000000 0.010642094393187763
1047100000 0.005267732234314416
1047200000 0.011038558866145213
1047300000 0.00902171792825063
best so far: 0
type: [4, 1, 1, 17,